In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score, roc_auc_score

In [2]:
df = pd.read_csv('../../data/crawler/unified-events-statistics.csv')
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result
0,36864,ESPORTSTMNT02_2556988,19,1,2,1,3,24,2,23,...,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0,1
1,36865,ESPORTSTMNT05_2520933,19,21,1,22,25,24,20,21,...,8,0.38,3.2,21,0.57,3.3,6,0.83,4.1,0
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,0
3,36867,ESPORTSTMNT02_2557426,20,0,21,20,21,5,23,1,...,15,0.67,3.9,34,0.74,7.3,2,0.00,2.0,0
4,36868,ESPORTSTMNT02_2557463,19,1,2,1,2,4,20,24,...,5,0.40,2.4,2,1.00,24.0,1,1.00,6.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4584,44417,ESPORTSTMNT02_3080864,19,1,21,1,2,4,1,5,...,29,0.69,5.8,55,0.71,5.6,7,0.57,2.1,1
4585,44418,ESPORTSTMNT02_3080870,21,19,20,2,22,1,4,3,...,39,0.77,4.6,5,0.60,6.6,7,0.86,6.3,1
4586,44419,ESPORTSTMNT02_3080871,0,20,21,5,1,21,22,1,...,30,0.67,5.4,87,0.64,4.7,11,0.55,2.8,1
4587,44420,ESPORTSTMNT02_3080872,19,20,21,1,2,5,4,3,...,27,0.67,4.3,14,0.57,3.5,1,1.00,20.0,1


In [3]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,event1,event2,event3,event4,event5,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,19,1,2,1,3,1,1.00,2.0,1,1.00,...,0.0,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0
1,19,21,1,22,25,5,0.60,5.6,2,0.00,...,3.5,8,0.38,3.2,21,0.57,3.3,6,0.83,4.1
2,19,2,20,24,2,0,0.00,0.0,1,1.00,...,0.0,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2
3,20,0,21,20,21,23,0.61,3.6,5,0.20,...,4.2,15,0.67,3.9,34,0.74,7.3,2,0.00,2.0
4,19,1,2,1,2,3,1.00,9.8,2,1.00,...,4.7,5,0.40,2.4,2,1.00,24.0,1,1.00,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4584,19,1,21,1,2,5,1.00,4.8,42,0.88,...,3.8,29,0.69,5.8,55,0.71,5.6,7,0.57,2.1
4585,21,19,20,2,22,21,0.52,2.5,22,0.59,...,3.8,39,0.77,4.6,5,0.60,6.6,7,0.86,6.3
4586,0,20,21,5,1,11,1.00,9.5,8,0.63,...,3.6,30,0.67,5.4,87,0.64,4.7,11,0.55,2.8
4587,19,20,21,1,2,34,0.47,2.9,1,1.00,...,13.3,27,0.67,4.3,14,0.57,3.5,1,1.00,20.0


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

parameters = {
    "loss":["deviance","log_loss","exponential"],
    "learning_rate": [0.025,0.05,0.1],
    "min_samples_split": [1,3,8],
    "min_samples_leaf": [1,3,8],
    "max_depth":[1,3,8],
    "max_features":["log2","auto","sqrt"],
    "criterion": ["friedman_mse","squared_error", "mse"],
    "subsample":[0.1,0.5,1.0],
    "n_estimators":[10,100,500]
    }

gbc = GradientBoostingClassifier(random_state=42)

cv = GridSearchCV(gbc,parameters,cv=3, n_jobs=-1)
cv.fit(X_train,y_train)

KeyboardInterrupt: 

In [ ]:
def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')

In [ ]:
display(cv)

In [ ]:
gbOpt= GradientBoostingClassifier(random_state=42,criterion='friedman_mse',learning_rate=0.025,loss='deviance',max_depth=3,max_features='log2',min_samples_leaf=1,min_samples_split=2, n_estimators=60,subsample=1.0)
gbOpt.fit(X_train, y_train)

scores_list = []
scores_list.append({
    'Balanced Accuracy': balanced_accuracy_score(y_test,gbOpt.predict(X_test)),
    'Precision':  precision_score(y_test,gbOpt.predict(X_test)),
    'Recall': recall_score(y_test,gbOpt.predict(X_test)),
    'F1-Score': f1_score(y_test,gbOpt.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,gbOpt.predict(X_test))
})
scores = pd.DataFrame(scores_list)

In [ ]:
scores